In [30]:
import pymongo
import pandas as pd
import random
import json
from os.path import join,dirname
import os
from pymongo import MongoClient
import random
from datetime import date, timedelta, datetime
DATEFORMAT = "%Y-%m-%d"

client = MongoClient('localhost', 27017)
db = client['aion']
collection = db['employee']

In [69]:
ROOT_DIR = os.path.abspath(os.curdir)
filename =join(ROOT_DIR, 'data/employee_mock_data.json')
if filename:
    with open(filename, 'r') as f:
        employees = json.load(f)
print(employees[0:2])
# bias salary by age and by gender
employee_list = []
for key,employee in enumerate(employees):
    employee_list.append(employee['name'])
    if employee['hourly_rate'] < 0:
        employee['hourly_rate'] = 20
    if employee['gender'].lower() == 'female':
        employee['hourly_rate'] *= random.choice([1.3, 1.6, 1.4, 1.1])
    dt = datetime.strptime(employee['dob'],DATEFORMAT)
    employee['dob'] = datetime(dt.year,dt.month,dt.day,0,0,0)
    employee['hourly_rate'] += round((datetime.today() - employee['dob']).days/900,2)
print(employees[0:2])

#collection.insert_many(employees)


[{'id': 1, 'name': 'Cindra Serotsky', 'department': 'Marketing', 'title': 'Senior Cost Accountant', 'gender': 'Female', 'dob': '1993-06-09', 'hourly_rate': 14}, {'id': 2, 'name': 'Nataline Marples', 'department': 'Business Development', 'title': 'Actuary', 'gender': 'Female', 'dob': '1996-09-07', 'hourly_rate': 20}]
[{'id': 1, 'name': 'Cindra Serotsky', 'department': 'Marketing', 'title': 'Senior Cost Accountant', 'gender': 'Female', 'dob': datetime.datetime(1993, 6, 9, 0, 0), 'hourly_rate': 25.910000000000004}, {'id': 2, 'name': 'Nataline Marples', 'department': 'Business Development', 'title': 'Actuary', 'gender': 'Female', 'dob': datetime.datetime(1996, 9, 7, 0, 0), 'hourly_rate': 41.19}]


In [83]:
import numpy as np
from scipy.stats import gamma

# fill projects
collection = db['project_type']
dct_type = {
    'type':['research','innovation','audit','reconciliation','conference','manufacturing',
            'construction']
}
#df_type = pd.DataFrame.from_dict(dct_type)
#data_json = json.loads(df_type.to_json(orient='records'))
#collection.insert_many(data_json)

types = dct_type['type']

collection_workers = db['project_task']
tasks_list = []
project_lists = []
for idx in range(1,351):
    dct = {}
    # generate project type
    name = 'Project '+str(idx)
    dct['name'] = name
    dct['type'] = random.choice(types)
    s = random.choice(['closed','closed','open'])
    # generate dates
    first_date = datetime(2013,1,1,0,0,0)
    sp = first_date + timedelta(days=random.choice(list(range(0,1925))))
    dct['startdate_proposed'] = sp
    proj_len = random.choice(list(range(5,150)))
    ep = sp + timedelta(days=proj_len)
    dct['enddate_proposed'] = ep
    
    s = 'closed'
    if ep > datetime.today():
        s = 'open'
    dct['status'] = s

    delay = int(gamma.rvs(a=0.5, size=1))
    if delay < 0:
        delay = 0
    sa = sp + timedelta(days=delay)
    dct['startdate_actual'] = sa
    # enddate delay
    if s == 'closed':
        delay = int(gamma.rvs(a=10, size=1))
        delay *= random.choice([-1,1,1])
        
        dct['enddate_actual'] = ep + timedelta(days=delay) 
    else:
        dct['enddate_actual'] = datetime(2020,1,1,0,0,0)
    
    project_lists.append(dct)
    
    ####################### tasks
    emps = int(gamma.rvs(a=20, size=1))
    for emp in range(0,emps):
        dct_tasks= {}
        dct_tasks['project'] =  name
        task_len = proj_len * 24 *.25# hours
        dct_tasks['employee'] = random.choice(employee_list)
        
        start = sa + timedelta(days=random.choice(list(range(0,int(proj_len*.8)))))
        dct_tasks['start'] = start
        end = start + timedelta(hours= int(gamma.rvs(a=task_len, size=1)))
        if end > sa:
            end = sa
        dct_tasks['end'] = end
        tasks_list.append(dct_tasks)
        
collection_project.insert_many(project_lists)
collection_workers.insert_many(tasks_list)

print(df.head(30))
print(df_tasks.head(40))

          name            type  status enddate_proposed startdate_proposed  \
0    Project 1      innovation  closed       2014-03-23         2014-02-06   
1    Project 2      innovation  closed       2013-12-17         2013-10-03   
2    Project 3        research  closed       2017-08-04         2017-07-28   
3    Project 4   manufacturing  closed       2013-07-05         2013-04-23   
4    Project 5    construction  closed       2015-10-05         2015-07-15   
5    Project 6    construction  closed       2013-08-05         2013-04-18   
6    Project 7  reconciliation  closed       2018-04-25         2017-12-15   
7    Project 8           audit  closed       2017-03-02         2017-01-19   
8    Project 9      conference  closed       2017-07-21         2017-04-27   
9   Project 10    construction  closed       2015-07-13         2015-02-21   
10  Project 11           audit  closed       2017-07-01         2017-06-16   
11  Project 12           audit  closed       2016-02-06         